# 固有値と固有ベクトルの意味を理解する(2022/01/18)
---
## 概要
---
本notebookでは固有値と固有ベクトルについて調査し，その性質についてまとめる．

簡単にまとめると，以下の3点：
- $n\times n$である正方行列$A$について，固有値・固有ベクトルが定義される．
- 行列$A$を線形変換の作用素だととらえると，固有ベクトルは元のデータのうち「変換によって向きが変わらない部分」という意味になる．例えば画像における線形変換(回転，拡大縮小等)では，固有ベクトル方向の変化は起こらない(定数倍される可能性はある)．
- データサイエンスの分野では，固有値と固有ベクトルは主成分分析等の用途に用いられる．主成分分析はデータの「要約」を行う代表的な手法である．

## 固有値・固有ベクトル
---
任意の$n\times n$の正方行列$A\in \mathbb{R}^{n \times n}$について：

\begin{eqnarray}
Ax = \lambda x 
\end{eqnarray}

が成り立つとき，ベクトル$x$を「行列$A$の固有ベクトル」，定数$\lambda$を「行列$A$の固有値」と呼ぶ．固有ベクトル$x$には，零ベクトル$\mathbf{0}$は含まれない．

この式を見ると，固有ベクトル$x$に対して行列$A$で線形変換を行ったとしても，結果は固有値$\lambda$倍(定数倍)にしかならないという性質を表していることが分かる．

例えば，ある画像に対して線形変換(拡大縮小，回転，剪断等)を行った場合，固有ベクトルの方向には高々定数倍の影響しかないということである．画像処理以外でも，例えば地球のような回転体について，固有ベクトルは自転軸と同じ向きを向いている．線形変換以外にも，物体の運動などについて固有ベクトルを捉えられるのは非常に面白い(固有値・固有ベクトルは分野問わず様々な機会に見ることがあるので，非常に重要な概念)．

### 固有値・固有ベクトルの導出
ある正方行列$A$から実際に固有値・固有ベクトルを導出する場合，以下のような方程式を解く必要がある：
\begin{eqnarray}
det(A - \lambda I) = 0
\end{eqnarray}

$I$は単位行列を表す．これはつまり，行列$A-\lambda I$の行列式がゼロとなる$\lambda$の値を求める方程式である．これを「固有方程式(特性方程式)」と呼ぶ．固有方程式を解いて$\lambda$の値を求めたのち，固有ベクトルの定義式に値を代入することで，芋づる式に固有ベクトルも求めることができる．

扱う行列によって$\lambda$の解が複数求まる可能性もある．その個数は$n$個以下になる．

なお，現実的に手計算で導出するのは$n=2,3$あたりが限界だと思う．サイズの大きい行列の固有値・固有ベクトルの導出には，計算機の力を借りるのが妥当だろう．

### Pythonを用いたシミュレーション
Numpyには，行列を引数として与えることで固有値・固有ベクトルを返してくれる関数numpy.linalg.eigが用意されている．

In [1]:
import numpy as np

# 適当な正方行列を定義する
matrix = np.array([
    [1, 3],
    [3, 1]
])

eigen_values, eigen_vectors = np.linalg.eig(matrix)

for eigen_value, eigen_vector in zip(eigen_values, eigen_vectors):
    print('eigen_value:', eigen_value)
    print('eigen_vector:', eigen_vector)

eigen_value: 3.9999999999999996
eigen_vector: [ 0.70710678 -0.70710678]
eigen_value: -2.000000000000001
eigen_vector: [0.70710678 0.70710678]


## 主成分分析
---
固有値・固有ベクトルの応用先として，データサイエンスでは主成分分析が挙げられる．分散が大きくなるようにデータを見れば，たくさんの情報が得られるはずだという考え方に基づく分析手法である．

これはデータの分散共分散行列に対して固有値・固有ベクトルを求めることによって実現される．分散(固有値)が最も大きくなる軸(固有ベクトル)からデータを捉えなおすことで，データ点を特徴づける要素が何か，分析しやすくなる．

また，主成分分析の効果として次元削減が挙げられる．データ点の次元を減らすことで，機械学習時の過学習の抑制や，2次元・3次元での可視化といったメリットが得られる．

注意点として，主成分分析は特徴量のスケールの大きさの影響を強く受ける点が挙げられる．「分散が最も大きくなる方向に新たな軸を考える」という操作なので，各次元のスケールが大きく異なっている場合には一部の特徴量の影響を強く反映する場合がある．例えば，あるデータが「0から100」のオーダーをもつ特徴量と，「-1000~1000」のオーダーを持つ特徴量からなる場合，「-1000から1000」を取りうる値の分散が大きくなり，主成分分析にも大きな影響を与える．

以上の注意点から，主成分分析を行う際はあらかじめデータを標準化(平均0，分散1に補正する)しておくか，相関行列から固有値・固有ベクトルを求めるほうが良い．

### 主成分分析と固有値問題のつながり
ここでは，あるデータ点を線形変換する操作について，分散を最大化する目的に沿うと分散共分散行列の固有値問題に帰着できることを簡易に説明する．

ここでは$n$個のデータからなる2次元のデータセット$D=\{(x_1, y_1),(x_2, y_2),...,(x_i, y_i),...,(x_n, y_n)\}$があるとして，これを主成分分析することを考える．

まず，データ点を別の点zへ線形変換する．これは：

\begin{eqnarray}
z_i = \alpha x_i + \beta y_i
\end{eqnarray}

と表現できる．$\alpha, \beta$はそれぞれ定数である．

このとき，変換先の$z$座標について，データ$D$の分散$s_z^2$を考えると：

\begin{eqnarray}
s_z^2 &=& \frac{1}{n}\sum^{n}_{i=1}(z_i - \bar{z})^2 \\
&=& \frac{1}{n}\sum^{n}_{i=1}[(\alpha x_i + \beta y_i) - (\alpha\bar{x} + \beta\bar{y})]^2 \\
&=& \frac{1}{n}\sum^{n}_{i=1}[\alpha(x_i - \bar{x}) + \beta(y_i - \bar{y})]^2 \\
&=& \frac{1}{n}\sum^{n}_{i=1}[\alpha^2(x_i - \bar{x})^2 +\alpha\beta(x_i - \bar{x})(y_i - \bar{y}) + \beta^2(y_i - \bar{y})^2] \\
&=& \alpha^2 \times \frac{1}{n}\sum^{n}_{i=1}(x_i - \bar{x})^2 + \alpha\beta \times \frac{1}{n}\sum^{n}_{i=1}(x_i - \bar{x})(y_i - \bar{y}) + \beta^2 \times \frac{1}{n}\sum^{n}_{i=1}(y_i - \bar{y})^2 \\
&=& \alpha^2s_x^2 + \alpha\beta s_{xy} + \beta^2s_y^2
\end{eqnarray}

となる．ここで，$s_x^2$は変数$x$の分散，$s_y^2$は変数$y$の分散，$s_{xy}$は変数$x, y$の共分散である．

分散$s_z$を最大化するために，ラグランジュの未定乗数法を用いる．

制約として，

### 寄与率
### Pythonを用いたシミュレーション

## 参考文献
---

[固有値，Wikipedia](https://ja.wikipedia.org/wiki/%E5%9B%BA%E6%9C%89%E5%80%A4)

[固有値，固有ベクトルの求め方，geisya.or.jp](https://www.geisya.or.jp/~mwm48961/linear_algebra/eigenvalue2.htm)

[固有値とその応用(pptスライド)](http://www.akita-pu.ac.jp/system/elect/ins/kusakari/japanese/teaching/Old/LinearAlgebra/2009/note/10.pdf)

[固有値と固有ベクトル(pdfファイル)](http://www.osakac.ac.jp/labs/mandai/writings/sd2-02m3-f.pdf)

[固有値の定義と求め方をていねいに～計算の手順～，数学の景色](https://mathlandscape.com/eigenvalue/)

[numpy.linalg.eig，Numpy v1.22 Manual](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html)

[主成分分析とは，インテージ](https://www.intage.co.jp/glossary/401/)

[主成分分析，Wikipedia](https://ja.wikipedia.org/wiki/%E4%B8%BB%E6%88%90%E5%88%86%E5%88%86%E6%9E%90)

[主成分分析とは？ R を使った分析例や因子分析との違いを解説，Quest](https://quest-research.co.jp/research/principal-component-analysis)

[主成分分析，統計科学研究所](https://statistics.co.jp/reference/software_R/statR_9_principal.pdf)

[主成分分析とは何なのか、とにかく全力でわかりやすく解説する，CORE CONCEPT](https://recruit.cct-inc.co.jp/tecblog/machine-learning/pca-kaisetsu/)

[Rと主成分分析](https://www.cis.doshisha.ac.jp/mjin/R/24/24.html)

[主成分分析とは？因子分析との違いや事例を初心者にも分かりやすく解説，GMO RESERCH](https://gmo-research.jp/research-column/principal-component-analysis)

[主成分分析の考え方，Logistics of Blue](https://logics-of-blue.com/principal-components-analysis/)

[分散共分散行列の定義と性質，高校数学の美しい物語](https://manabitimes.jp/math/1020)

[相関行列の定義と分散共分散行列との関係，高校数学の美しい物語](https://manabitimes.jp/math/1294)

[座標平面上における回転の公式，具体例で学ぶ数学](https://mathwords.net/heimenkaiten)

[行列と一次変換](https://www.geisya.or.jp/~mwm48961/linear_algebra/transform1.htm)